In [29]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

In [2]:
df = pd.read_csv('../Health_cleaned_income_delta.csv', low_memory=False)
df.shape

(170761, 63)

In [3]:
columns_to_int = ['rmstat', 'ragender', 'rahispan', 'raracem', 'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe',
        'rdiabe', 'rcancre', 'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp', 'rhspnit', 'oop_spend',
        'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten', 'index_wave', 'insured_gov', 'uninsured',
        'retired', 'collegeplus', 'year', 'inter_year', 'year_of_birth', 'without_work', 'n_jobs', 'broken']

In [4]:
def to_int(el):
    try:
        return int(el)
    except ValueError:
        return el

In [5]:
for col in columns_to_int:
    df[col] = df[col].apply(to_int)

In [6]:
marriage_groups = ['married', 'separated', 'dead spouse', 'never married', 'divorced']
age_groups = ['young', 'middle', 'old', 'dead outside']

In [7]:
rmstat_df = df[(df['rmstat'] != '.m') & (df['rhltc'] != '.m')]

In [13]:
rmstat_df['rmstat'].unique()

array([1, 4, 7, 5, 3, 8, 2, 6], dtype=object)

In [34]:
def calculate_confidence_interval(df, group_name, groups, column, confidence=0.99):
    errors = []

    for group in groups:
        group_df = df[df[group_name] == group]
        n = len(group_df)
        mean, se = group_df[column].mean(), stats.sem(group_df[column])
        h = se * stats.t._ppf((1 + confidence) / 2., n-1)
        errors.append(h)
    
    return errors

In [76]:
colors = ['#3367f5', '#fc4747', '#00CC96', '#ed47fc', '#fcab47', '#41b9fa', '#b447fc', '#47fc83']

In [151]:
fig = go.Figure()

groups = [x for x in range(1, 9)]

errors = calculate_confidence_interval(rmstat_df, 'rmstat', groups, 'rhltc')
tmp = rmstat_df[rmstat_df['ragey_b'] <= 67]

group_data = tmp.groupby('rmstat')['rhltc'].mean()
all_mean_rhltc = tmp['rhltc'].mean()

fig.add_trace(go.Bar(
    x=group_data.index,
    y=group_data.values,
    error_y=dict(type='data', array=errors, visible=True),
    width=[0.6 for _ in range(8)],
    texttemplate=[f'{x:.2f}' for x in tmp.groupby('rmstat')['ragey_b'].mean().reindex(groups).values],
    textposition='outside',
    marker_color=colors
))

fig.update_layout(
    title='Среднее изменение здоровья в разных семейных статусах',
    xaxis_title='Семейный статус',
    yaxis_title='Среднее значение rhltc',
    width=900, height=600
)

fig.update_yaxes(range=[-0.5, 0])

fig.add_hline(
    y=all_mean_rhltc,
    line_dash='dash',
    line_color='black',
    line_width=1,
)

fig.add_annotation(x=7.5, y=-0.14,
    text=f'Среднее по всем группам: {all_mean_rhltc:.2f}',
    showarrow=True,
    arrowhead=1,
    ay=120)

fig.show()

todo: на график выше добавить средний возраст в каждый столбец, графически выделить плохие и хорошие группы

In [ ]:
rmstat_df['ragender'].replace({1: 'male', 2: 'female'}, inplace=True)

In [130]:
rmstat_df.columns

Index(['hhidpn', 'rmstat', 'ragender', 'rahispan', 'raracem', 'riwbegy',
       'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe', 'rdiabe', 'rcancre',
       'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp',
       'rhspnit', 'oop_spend', 'riearn', 'ripena', 'siearn', 'sipena', 'rcovr',
       'rcovs', 'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten',
       'index_wave', 'insured_gov', 'uninsured', 'retired', 'collegeplus',
       'year', 'cpi', 'rgov', 'sgov', 'inter_year', 'year_of_birth',
       'total_work_income', 'total_pension_income', 'total_gov_income',
       'total_income', 'without_work', 'n_jobs', 'riearn_delta',
       'siearn_delta', 'ripena_delta', 'sipena_delta', 'rgov_delta',
       'sgov_delta', 'total_work_income_delta', 'total_pension_income_delta',
       'total_gov_income_delta', 'total_income_delta', 'broken',
       'marriage_group', 'age_group'],
      dtype='object')

In [148]:
y_ticks = [-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2]

mean_in_group = rmstat_df['rhltc'].mean()

fig = go.Figure()

tmp = rmstat_df[rmstat_df['marriage_group'] == 'married']
fig.add_trace(go.Bar(name='Married',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups),
    # marker_color='rgb(85, 177, 242)'
    texttemplate=[f'{x:.2f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    # texttemplate=['text1', 'text2', 'text3', 'text4', 'text5'],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'separated']
fig.add_trace(go.Bar(name='Separated',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.2f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'dead spouse']
fig.add_trace(go.Bar(name='Dead Spouse',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.2f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'never married']
fig.add_trace(go.Bar(name='Never Married',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.2f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'divorced']
fig.add_trace(go.Bar(name='Divorced',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.2f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

fig.update_layout(
    barmode='group',
    width=1200, height=1000,
    title=f'Среднее изменение здоровья по полу и группе возраста в группе брака',
    xaxis_title='Группа возраста',
    yaxis_title='Среднее изменение здоровья',
    yaxis = dict(
        tickmode = 'array',
        tickvals = y_ticks,
        ticktext = y_ticks
    )
)
fig.update_yaxes(range=[-1, 0.2])

# fig.add_hline(
#     y=mean_in_group,
#     line_dash='dash',
#     line_color='black',
#     line_width=1,
#     annotation_text=f'Среднее по группе: {mean_in_group:.2f}',
#     annotation_position='bottom left'
# )

fig.show()

In [152]:
def hypothesis_check(t, p): 
    if (abs(t) > 2.59) and (p < 0.01): 
        print('Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%') 
    elif (abs(t) > 1.9667) and (p < 0.05): 
        print('Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 95%') 
    else: 
        print('Не удалось отвергнуть H0')

In [153]:
df_married = rmstat_df[rmstat_df['marriage_group'] == 'married']
df_separated = rmstat_df[rmstat_df['marriage_group'] == 'separated']
df_dead = rmstat_df[rmstat_df['marriage_group'] == 'dead spouse']
df_never = rmstat_df[rmstat_df['marriage_group'] == 'never married']
df_divorced = rmstat_df[rmstat_df['marriage_group'] == 'divorced']

In [158]:
t, p = stats.ttest_ind(df_married['rhltc'], df_separated['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_separated['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_separated['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди все сложно =', df_separated['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%
ddof = 9997
t = 2.740833856309917
p-value = 0.0061581371253321025
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди все сложно = -0.14367651276168628


In [155]:
t, p = stats.ttest_ind(df_married['rhltc'], df_dead['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_dead['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_dead['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди потерявших супруга =', df_dead['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%
ddof = 85035
t = 33.60493654157091
p-value = 1.269130988922816e-244
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди потерявших супруга = -0.26026261688440877


In [159]:
t, p = stats.ttest_ind(df_married['rhltc'], df_never['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_never['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_never['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди никогда не женатых =', df_never['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 95%
ddof = 17643
t = 2.5701908229720227
p-value = 0.01019058537571036
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди никогда не женатых = -0.13258561986781495


In [157]:
t, p = stats.ttest_ind(df_married['rhltc'], df_divorced['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_divorced['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_divorced['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди разведенных =', df_divorced['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%
ddof = 41241
t = 6.2136080893496475
p-value = 5.300175988549461e-10
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди разведенных = -0.14776958458056508
